In [ ]:
# --- Notebook 必备: 交互式后端 ---
%matplotlib widget

import numpy as np
import h5py
import ipywidgets as w
import matplotlib.pyplot as plt
from pytransform3d.plot_utils import make_3d_axis
from pytransform3d.transformations import plot_transform, transform_from
from pytransform3d.rotations import matrix_from_quaternion

# === 配置你的 HDF5 路径 & key ===
HDF5_PATH = "/home/ls/Desktop/tf_test/base_action_data3/Data/Data.hdf5"
POSE_KEY = "action/rightHand/pose"
QUAT_KEY = "action/rightHand/quaternion"

# === 读取数据 ===
def load_right_hand(path, pose_key=POSE_KEY, quat_key=QUAT_KEY):
    with h5py.File(path, "r") as f:
        if pose_key not in f or quat_key not in f:
            raise KeyError(f"缺少所需数据集：{pose_key} 或 {quat_key}")
        P = np.asarray(f[pose_key])      # 期望形状: (N, 3?) 或 (N, >=3)
        Q = np.asarray(f[quat_key])      # 期望形状: (N, 4) （四元数）
    P = P.reshape(len(P), -1)
    if P.shape[1] >= 3:
        pos = P[:, :3].astype(float)
    else:
        pos = np.zeros((len(P), 3), dtype=float)
    Q = Q.reshape(len(Q), -1).astype(float)
    if Q.shape[1] < 4:
        raise ValueError(f"四元数维度不足: {Q.shape}")
    N = min(len(pos), len(Q))
    return pos[:N], Q[:N]

pos_all, quat_all = load_right_hand(HDF5_PATH)
N = len(pos_all)
# 估个显示范围
extent = float(np.nanmax(np.abs(pos_all))) if pos_all.size else 1.0
extent = max(0.5, min((extent if np.isfinite(extent) else 1.0) * 1.5, 5.0))

# === 小工具 ===
def mirror4(axis='z'):
    """返回 4x4 镜像矩阵 S，翻转 x/y/z 中的一轴；用于把 LH 世界嵌入到 RH 渲染坐标。"""
    S = np.eye(4)
    i = {'x':0, 'y':1, 'z':2}[axis]
    S[i, i] = -1.0
    return S

def reorder_quat(q4, order='wxyz'):
    """将四元数重排到 wxyz（pytransform3d 默认）"""
    q4 = np.asarray(q4, dtype=float)
    if order == 'wxyz':
        return q4
    elif order == 'xyzw':
        x, y, z, w = q4
        return np.array([w, x, y, z], dtype=float)
    else:
        return q4

def normalize_quat(q4):
    n = np.linalg.norm(q4)
    return q4 if n == 0 else q4 / n

# === 交互控件 ===
frame = w.IntSlider(0, min=0, max=N-1, step=1, description='frame', continuous_update=True)
play  = w.Play(interval=40, value=0, min=0, max=N-1, step=1, description="Press play")
w.jslink((play, 'value'), (frame, 'value'))

flip_axis   = w.Dropdown(options=['x','y','z'], value='z', description='LH flip')  # 左手系由哪轴翻转得到（常用 z）
quat_order  = w.ToggleButtons(options=[('wxyz','wxyz'), ('xyzw','xyzw')], value='wxyz', description='quat')
norm_quat   = w.Checkbox(value=True, description='normalize quat')
show_world  = w.Checkbox(value=True, description='show world')
axis_len    = w.FloatSlider(0.20, min=0.05, max=0.60, step=0.01, description='axis_len')

# === 画布 ===
ax = make_3d_axis(ax_s=1.0)
ax.set_title('RightHand in LEFT-handed world (S·T_LH)')
ax.set_xlim(-extent, extent); ax.set_ylim(-extent, extent); ax.set_zlim(-extent, extent)
ax.set_box_aspect([1,1,1])

def draw(_=None):
    ax.cla()
    ax.set_title('RightHand in LEFT-handed world (S·T_LH)')
    ax.set_xlim(-extent, extent); ax.set_ylim(-extent, extent); ax.set_zlim(-extent, extent)
    ax.set_box_aspect([1,1,1])

    i = int(frame.value)
    t_lh = pos_all[i].astype(float)                 # 左手系平移（直接用数据）
    q    = reorder_quat(quat_all[i], quat_order.value)
    if norm_quat.value:
        q = normalize_quat(q)

    # 在 LH 世界中构旋转矩阵；左/右手系的“旋转方向”由世界基底决定，这里后续会整体用 S 嵌入
    R_lh = matrix_from_quaternion(q)                # 期望 q 为 wxyz
    T_lh = transform_from(R_lh, t_lh)

    S = mirror4(flip_axis.value)
    if show_world.value:
        plot_transform(ax, A2B=S, s=0.25, strict_check=False)   # 世界轴（左手）
    plot_transform(ax, A2B=S @ T_lh, s=float(axis_len.value), strict_check=False)  # 物体轴（左手）

    ax.figure.canvas.draw_idle()

for wd in [frame, flip_axis, quat_order, norm_quat, show_world, axis_len]:
    wd.observe(draw, names='value')

ui = w.VBox([
    w.HBox([frame, play]),
    w.HBox([flip_axis, quat_order, norm_quat]),
    w.HBox([axis_len, show_world]),
])
display(w.VBox([ui, ax.figure.canvas]))

draw()
# 红色x 绿色y 蓝色z